[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert/blob/main/part4/ch8/p8_type2.ipynb)

In [46]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.4 MB 2.0 MB/s eta 0:00:01
   --------- ------------------------------ 0.4/1.4 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 13.2 MB/s eta 0:00:00


In [53]:
# 문제정의
# 평가: MAE
# target: TotalCharges
# 최종파일: result.csv(컬럼 1개 pred)

# 라이브러리 및 데이터 불러오기
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/churn_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch8/churn_test.csv")

# 탐색적 데이터 분석(EDA)
print(train.shape)
print(test.shape)
print("=========================================")

print(train.info())
print(test.info())
print("=========================================")

print(train.describe())
print(test.describe())
print("=========================================")

print(train.isnull().sum().sum())
print(test.isnull().sum().sum())
print("=========================================")

print(train['customerID'].nunique())
print(test['customerID'].nunique())

(4116, 19)
(1764, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4116 entries, 0 to 4115
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        4116 non-null   object 
 1   gender            4116 non-null   object 
 2   SeniorCitizen     4116 non-null   int64  
 3   Partner           4116 non-null   object 
 4   Dependents        4116 non-null   object 
 5   tenure            4116 non-null   int64  
 6   PhoneService      4116 non-null   object 
 7   MultipleLines     4116 non-null   object 
 8   InternetService   4116 non-null   object 
 9   OnlineSecurity    4116 non-null   object 
 10  OnlineBackup      4116 non-null   object 
 11  DeviceProtection  4116 non-null   object 
 12  TechSupport       4116 non-null   object 
 13  StreamingTV       4116 non-null   object 
 14  StreamingMovies   4116 non-null   object 
 15  Contract          4116 non-null   object 
 16  PaperlessBilling  41

In [54]:
# 데이터 전처리
train = train.drop('customerID', axis=1)
test = test.drop('customerID', axis=1)
target = train.pop('TotalCharges')

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns
print(cols)

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])
    
# 검증데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.1, random_state= 0)

# 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

# MAE
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, y_pred))

# LightGBM
import lightgbm as lgb
lgb = lgb.LGBMRegressor(random_state=0, verbose=-1)
lgb.fit(x_train, y_train)
y_pred = lgb.predict(x_test)
print(mean_absolute_error(y_test, y_pred))

# 최종 제출 파일
y_pred = rf.predict(x_test)

result = pd.DataFrame({
    'pred':y_pred
})
result.to_csv('result.csv', index=False)

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')
928.6802749150485
938.2570752478721


In [58]:
# 1. pred 행의 수
print(result.shape)

# 2. 생성한 csv 확인
print(result)

(412, 1)
           pred
0    3493.36570
1    3484.46130
2    4089.08590
3    3853.32650
4    3620.95690
..          ...
407   586.19660
408  2562.65965
409  4797.74050
410  2253.92040
411  3301.83460

[412 rows x 1 columns]
